In [ ]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import json
import os
import sys
sys.path.append("../lib")
from dataUtils import *
import cPickle as pickle
import cv2
from collections import Counter
from scipy.interpolate import UnivariateSpline


In [ ]:
#ClassFile = "../Logs/classes.json"
ClassFile = "../Logs/english_label.txt"

#vineScores = "../Logs/sampled_vine_sentibank_final.csv"
vineScores = "../Logs/MVSO_fine_vine_probs.csv"

#selfieScores = "../Logs/selfieSentibankProbs.csv"
sentibank_scores = "../Logs/sentibank_baseline_final.csv"

#vineANPs = "../Logs/sampled_vine_ANPS_final.pk"
vineANPs = "../Logs/MVSO_fine_vine_ANPs.pk"
#selfiePaths = "../Logs/selfiePaths.txt"

imageNetObjs = "../Logs/sampledvineImagenetObjs2015_1.pk"

#selfiePopularityFile = "../Logs/selfie_dataset.txt"

root = "../vinedata/Data/"

visitedList = "../Logs/sampledVids.data"

postFile = "../Logs/postsMapping.csv"

# sentimentFile = "../Logs/ANP_Sentiments.txt"

color_features = "../Logs/vine_features_ordered.csv"

sampled_img_list = "../Logs/sampled_sentibank_image.txt"

revisedSentimentFile = "../Logs/revised_ANP_sentiments.csv"

In [ ]:
#Load the Sentibank scores for Vines
vineProbs = np.loadtxt(vineScores, delimiter=',')


In [ ]:
#Load the Sentibank scores for Baseline
sentibank_baseline = []
#sentibank_baseline = np.loadtxt(sentibank_scores, delimiter=',')

In [ ]:
print vineProbs.shape

In [ ]:
def get_ANP_ID_List(pickleList , nameIdx):
    ANPs = []
    IDs = []
    for line in pickleList:
        ids = line.split(',')[0].split('/')[nameIdx].split('_')[0]
        IDs.append(ids)
        ANPs.append(line.split(',')[1])
    return IDs , ANPs

def get_vid_senti(pickle , index , nameidx):
    oldId = pickle[index].split(',')[0].split('/')[nameidx].split('_')[0]
    seqDict = dict()
    indexList = []
    sequence = pickle[index].split(',')[0].split('/')[nameidx].split('_')[1].split('.')[0]
    seqDict[int(sequence)] = str(pickle[index].split(',')[1])
    indexList.append(index)
    index+=1
    #print index
    while (index < len(pickle) and (pickle[index].split(',')[0].split('/')[nameidx].split('_')[0] == oldId)):
        sequence = pickle[index].split(',')[0].split('/')[nameidx].split('_')[1].split('.')[0]
        seqDict[int(sequence)] = str(pickle[index].split(',')[1])
        indexList.append(index)
        index += 1
    seqDict['indexList'] = indexList
    return seqDict , oldId , index

#This function maps each video with a dictionary entry that has list of all ANPS per frame
#and an index list to find them
def get_VID_ANP_List(pickle, nameidx):
    megaDict = dict()
    i = 0
    print len(pickle)
    while i < len(pickle):           
        subDict , postId , i = get_vid_senti(pickle , i , nameidx)
        megaDict[int(postId)] = subDict
    return megaDict


def pruneMegaDict(megadict , filterindices):
    filteredList = dict()
    for entry in megadict:
        commns = set(megadict[entry]['indexList']).intersection(filterindices)
        if len(commns) >= 6:
            filteredList[entry] = megadict[entry]
    return filteredList

def readJson(path):
    f = open(path)
    data = json.loads(f.read())
    return data


In [ ]:
image_list =[]
with open(sampled_img_list) as f:
    image_list = f.readlines()

true_labels = []
for line in image_list:
    label = line.split('/')[5]
    true_labels.append(label)
    
# f = open(ClassFile ,'r')
# sentibankClasses = json.load(f)
# f.close()

f = open(ClassFile ,'r')
sentibankClasses = f.readlines()
f.close()

In [ ]:
# sentibank_probs = np.zeros(len(sentibank_baseline))
# detected_labels = []
# for i in range(sentibank_probs.shape[0]):
#     sentibank_probs[i] = sentibank_baseline[i].max()
#     detected_labels.append(str(sentibankClasses[np.argmax(sentibank_baseline[i])]))
    
# print np.median(sentibank_probs), sentibank_probs.var(), sentibank_probs.mean()

In [ ]:
maxVecs = np.zeros(len(vineProbs))
for i in range(len(vineProbs)):
    maxVecs[i] = np.max(vineProbs[i])
print np.mean(maxVecs) , np.median(maxVecs) , np.var(maxVecs)

In [ ]:
curatedProbs_index = []
for i in range(len(vineProbs)):
    if (vineProbs[i].max() > (np.mean(maxVecs))):
        curatedProbs_index.append(i)

        
print len(curatedProbs_index)

In [ ]:

vineList = readLists(vineANPs)
objList = readLists(imageNetObjs)
idList, anpList = get_ANP_ID_List(vineList , 6)


In [ ]:
print anpList[5]
print idList[1]

In [ ]:
print vineList[0].split(',')[0].split('/')[6].split('_')[0]
print vineList[0]

In [ ]:
sentidict0 = readSentiments()
sentidict = readRevisedSentiments()

In [ ]:
sentiArray = []
for k in sentidict:
    sentiArray.append(sentidict[k])
print len(sentiArray)

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(15, 10)
plt.hist(sentiArray, 1000, normed=1,histtype='step', cumulative=True,linewidth = 3.0)
plt.title("Distribution of ANP sentiments", fontsize = 20)
plt.xlabel("Sentiment value",fontsize = 15)
plt.ylabel("Frequency",fontsize = 15)
plt.grid()
plt.show()

In [ ]:
# f = open('sentibankANP.csv', 'wb')
# for k,v in sentidict0.iteritems():
#     f.write(k+','+str(v)+'\n')
#     #print k , v
# f.close()

In [ ]:
megaDict = get_VID_ANP_List(vineList , 6)


In [ ]:
# print megaDict[1281867050901532672]
# videoSentimentIndices = megaDict[1281867050901532672]['indexList']
# for i in videoSentimentIndices:
#     print vineList[i]

In [ ]:
filteredDict = pruneMegaDict(megaDict, curatedProbs_index)

In [ ]:
print len(filteredDict)


In [ ]:
len(megaDict)

In [ ]:
postIds = []
for line in vineList:
    arr = line.split('/')
    i = int(arr[6].split('_')[0])
    postIds.append(i)

In [ ]:
#Vectors for filtered videos
senti_matrix = np.zeros((len(filteredDict),12))
ANP_matrix = list()
print senti_matrix.shape

In [ ]:
i = 0
misses = 0
postIdFilteredList = []
for entry in filteredDict:
    postIdFilteredList.append(entry)
    ANPEntry = list()
    for j in range(1,13):
        
        if j in filteredDict[entry]:
            senti_matrix[i][j-1] = sentidict[filteredDict[entry][j]] \
            if (filteredDict[entry][j] in sentidict) \
            else sentidict[filteredDict[entry][j-1]]
            
            ANPEntry.append(filteredDict[entry][j])
        else:
            last = ANPEntry[-1]
            ANPEntry.append(last)
            senti_matrix[i][j-1] = senti_matrix[i][j-2]
            misses += 1
    ANP_matrix.append(ANPEntry)
    i += 1
print misses

In [ ]:
def intersect(a, b):
    result=[]
    for i in b:
        if isinstance(i,list):
            result.append(intersect(a,i))
        else:
            if i in a:
                 result.append(i)
    return result

def JaccardSim(l1 , l2):
    intersection = list(set(l1).intersection(l2))
    union = list(set(l1).union(l2))
    #intersection = intersect(l1 , l2)
    jaccard = float(len(intersection)/len(union))*1.0
    #jaccard = float(len(intersection)/(len(l1) + len(l2)))*1.0
    return jaccard

def similarityMatrix(ANPList):
    simMatrix = np.zeros((len(ANPList),len(ANPList)))
    for i in range(len(ANPList)):
        for j in range(len(ANPList)):
            jacSim = JaccardSim(ANPList[i],ANPList[j])
            simMatrix[i][j] = jacSim
    return simMatrix

def autocorr(x):
    result = np.correlate(x, x, mode='full')
    return result[result.size/2:]

def findAutocorr(sentimatrix):
    autocorr_matrix = []
    for i in range(senti_matrix.shape[0]):
        autocorr_matrix.append(list(autocorr(senti_matrix[i])))
    return np.asarray(autocorr_matrix)

In [ ]:
sampled = np.random.choice(len(senti_matrix) , 2999)
sampled_matrix = senti_matrix[sampled,:]
print sampled_matrix.shape

In [ ]:
autocorr_matrix = findAutocorr(sampled_matrix)

In [ ]:
print autocorr_matrix.mean(axis=0)

In [ ]:
Jaccard_Matrix = similarityMatrix(ANP_matrix)

In [ ]:
heatMap = Jaccard_Matrix
fig, ax = plt.subplots()
fig.set_size_inches(30, 30)
plt.imshow(heatMap[1000:2000,1000:2000], cmap='hot' , interpolation='nearest')

In [ ]:
#Get all post meta data 
with open(postFile, 'rb') as f:
        lines = f.readlines()

postDict = {}
postList = []
for line in lines:
    comp = line.split(',')
    postDict[comp[0]] = comp[1]
    postList.append(comp[1])

print len(postDict) , postList[1:10]

In [ ]:
selectedPosts = []
allPosts = []

for d in postList:
    dataRoot = "/datasets/sagarj/vineData/Dataset/Posts/" + d.strip()
    f = open(dataRoot , 'rb')
    data = json.loads(f.read())
    allPosts.append(data['data']['records'][0])

In [ ]:
print len(allPosts)

In [ ]:
filteredPosts = dict()
for k in filteredDict:
    for post in allPosts:
        if post['postId'] == k:
            filteredPosts[k] = post
print len(filteredPosts)

In [ ]:
import random
rand_smpl = [ allPosts[i] for i in sorted(random.sample(xrange(len(allPosts)), len(filteredPosts))) ]
print len(rand_smpl)

allLikes = []
allReposts = []
allLoops = []
for rec in rand_smpl:
    allLikes.append(rec['likes']['count'])
    allReposts.append(rec['reposts']['count'])
    allLoops.append(rec['loops']['count'])

In [ ]:
print np.min(allLikes) , np.max(allLikes) , np.min(allReposts) , np.max(allReposts)

In [ ]:
filLikes = []
filReposts = []
filLoops = []
filPosts = []
for k in filteredPosts:
    filLikes.append(np.log(filteredPosts[k]['likes']['count']+1))
    filReposts.append(np.log(filteredPosts[k]['reposts']['count']+1))
    filLoops.append(np.log(filteredPosts[k]['loops']['count']+1))
    filPosts.append(filteredPosts[k])

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(15, 10)
plt.hist(filLikes, 400, normed=1,histtype='step', cumulative=True,linewidth = 3.0,linestyle='dashed')
plt.hist(filReposts, 400, normed=1,histtype='step', cumulative=True,linewidth = 3.0)
plt.title("CDF for Likes and Repost", fontsize = 25)
plt.xlabel("Likes/repost count (Log scale)",fontsize = 25)
plt.ylabel("CDF",fontsize = 25)
plt.legend(['Likes' , 'Reposts'])
plt.show()


In [ ]:
sortedWRTLikesPosts = [x for (y,x) in sorted(zip(filLikes,filPosts))]
sortedWRTRepostsPosts = [x for (y,x) in sorted(zip(filReposts,filPosts))]

In [ ]:
print np.min(filLikes) , np.max(filLikes) , np.min(filReposts) , np.max(filReposts)

In [ ]:
print np.median(filLikes)
print np.median(filReposts)
print np.median(np.log(allLikes))
print np.median(np.log(allReposts))

In [ ]:
print np.corrcoef(filReposts,filLoops)
print np.corrcoef(filLikes,filLoops)
print np.corrcoef(filReposts,filLikes)

In [ ]:
simlist = []
simcount = []
dummy_Jaccard = Jaccard_Matrix.copy()
for i in range(dummy_Jaccard.shape[0]):
    sim = []
    count = 0
    for j in range(dummy_Jaccard.shape[1]):
        if dummy_Jaccard[i][j] == 1.0:
            sim.append(j)
            dummy_Jaccard[j][i] = 0.0
            count += 1
    simlist.append(sim)
    simcount.append(count)


In [ ]:
maxMatches = np.sort(simcount)[3000:3100]
matchIndexes = np.argsort(simcount)[3000:3100]

print maxMatches , matchIndexes
print np.argsort(simcount)[-5:]

In [ ]:
# fig, ax = plt.subplots()
# fig.set_size_inches(15, 10)
# plt.plot(simcount)


In [ ]:
import math
def dotproduct(v1, v2):
    return sum((a*b) for a, b in zip(v1, v2))

def length(v):
    return math.sqrt(dotproduct(v, v))

def angle(v1, v2):
    return math.acos(dotproduct(v1, v2) / (length(v1) * length(v2)))

def unit_vector(vector):
    """ Returns the unit vector of the vector.  """
    return vector / np.linalg.norm(vector)

def angle_between(v1, v2):
    """ Returns the angle in radians between vectors 'v1' and 'v2'::

            >>> angle_between((1, 0, 0), (0, 1, 0))
            1.5707963267948966
            >>> angle_between((1, 0, 0), (1, 0, 0))
            0.0
            >>> angle_between((1, 0, 0), (-1, 0, 0))
            3.141592653589793
    """
    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    return np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))

def topNColliniear(basis , vectors , topN):
    ang = []
    for vec in vectors:
        ang.append(angle_between(vec , basis))
    indexes = np.argsort(ang)
    print ang[indexes[0]],ang[indexes[-1]]
    print indexes[:topN]
    #returns the top N indexes with least angular value
    return indexes[:topN] , vectors[indexes[:topN]]

def getANPs(postDict , posts):
    ANP_list = []
    ids = [i['postId'] for i in posts]
    for postid in ids:
        ANP_list.append(postDict[postid])
    return ANP_list

def filterCluster(postids , posts , postDict, indexList):
    plist = []
    ANP_list = []
    ids = [postids[i] for i in indexList ]
    for postid in ids:
        plist.append(posts[postid])
        ANP_list.append(postDict[postid])
    return plist , ANP_list


In [ ]:
vidnumber = matchIndexes[35]
print np.sum(dummy_Jaccard[vidnumber,:])
matched = []
for i in range(len(dummy_Jaccard[vidnumber])):
    if dummy_Jaccard[vidnumber][i]== 1.0:
        matched.append(i)
print matched

In [ ]:
# index1 , topSentis1 = topNColliniear(principals[0], senti_matrix , 3) 
matchedPosts , ANPmatched  = filterCluster(postIdFilteredList,filteredPosts,filteredDict,matched)
#print ANPmatched
import io
import base64
from IPython.display import HTML

num = 2
url = matchedPosts[num]['videoUrl'].strip().split('/')[-1].split('?')[0]
print  matchedPosts[num]['videoUrl']
#print videosBasis1[num]
print url
videoPath = "/datasets/sagarj/vineData/Dataset/Videos/" + url
#print videoPath


video = io.open(videoPath, 'r+b').read()
encoded = base64.b64encode(video)
HTML(data='''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii')))

In [ ]:
from sklearn.svm import LinearSVR
clf = LinearSVR()

clf.fit(senti_matrix, filReposts) 
print clf.coef_

In [ ]:
print senti_matrix[:100]

In [ ]:
senti_vector = np.reshape(senti_matrix, senti_matrix.shape[0]*senti_matrix.shape[1])
fig, ax = plt.subplots()
fig.set_size_inches(15, 10)
plt.hist(senti_vector, 20, normed=1,histtype='step', cumulative=False,linewidth = 3.0)
plt.title("Distribution of all frames", fontsize = 20)
plt.xlabel("Sentiment value",fontsize = 15)
plt.ylabel("CDF",fontsize = 15)
plt.show()

In [ ]:
maxGraph = np.zeros(senti_matrix.shape[1])
minGraph = np.zeros(senti_matrix.shape[1])
varMaxGraph = np.zeros(senti_matrix.shape[1])
varMinGraph = np.zeros(senti_matrix.shape[1])
differenceList = []
id1 = []
id2 = []
for i in range(len(senti_matrix)):
    a = max(senti_matrix[i])
    b = min(senti_matrix[i])
    median_sent = np.median(senti_matrix[i])
    diff = a - b
    
    diff2 = a - median_sent
    if diff > 0.5:
        ind1 = senti_matrix[i].tolist().index(a)
        if ind1 < 2:
             id1.append(i)
        maxGraph[ind1] += 1
        ind2 = senti_matrix[i].tolist().index(b)
        if ind2 < 2:
             id2.append(i)
        minGraph[ind2] += 1
        differenceList.append(diff)
    
    
maxGraph = maxGraph/senti_matrix.shape[0]
minGraph = minGraph/senti_matrix.shape[0]
print maxGraph
print minGraph
print senti_matrix.shape
print np.sum(maxGraph),np.sum(minGraph)
print len(id1) , len(id2)

In [ ]:
c3 = list(set(id1).intersection(id2))
len(c3)

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(15, 10)
n, bins, patches = plt.hist(differenceList, 10, normed=0,histtype='step', cumulative=False,linewidth = 3.0)
plt.title("Distribution of difference between maximum and minimum sentiment in a video", fontsize = 20)
plt.xlabel("Sentiment value",fontsize = 15)
plt.ylabel("CDF",fontsize = 15)
plt.show()

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(15, 10)
plt.title("Frequency of finding max/min sentiment for a given video in nth 0.5 second interval (dropping constant sentiment videos)", fontsize = 15)
plt.xlabel("Nth 0.5 second interval into the video",fontsize = 15)
plt.ylabel("Frequency",fontsize = 15)
plt.plot(maxGraph,linewidth = 3.0)
plt.plot(minGraph,linewidth = 3.0)
plt.legend(['Max Value', 'Min Value'   
           ])
plt.show()

In [ ]:
from scipy.cluster.vq import kmeans,vq
from scipy.spatial.distance import minkowski

In [ ]:
centroid_list = []
id_list = []
for i in range(1 , 12):
    centroids,_ = kmeans(senti_matrix,i)
    idx,_ = vq(senti_matrix,centroids)
    centroid_list.append(centroids)
    id_list.append(idx)

In [ ]:
SSE_values = np.zeros(len(id_list))
for i in range(len(centroid_list)):
    for j in range(len(centroid_list[i])):
        vecs = senti_matrix[id_list[i]==j,:]
        #print vecs.shape
        cent = centroid_list[i][j]
        SSE_1 = 0.0
        for vec in vecs:
            SSE_1 = SSE_1 + minkowski(vec,cent,2)
        SSE_values[j] = SSE_values[j] + SSE_1
for i in range(len(SSE_values)):
    SSE_values[i] = SSE_values[i]/(i+1)

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(15, 15)
plt.rc('xtick', labelsize=20) 
plt.rc('ytick', labelsize=20)
x = np.arange(1,11)
plt.plot(x,SSE_values[:10] ,linewidth = 3.0)
plt.xlabel("Number of Clusters", fontsize = 25)
plt.ylabel("Mean Minkowski distance from cluster centroids", fontsize = 25)
plt.title("Cluster groupings for Sentiment vectors for Videos", fontsize = 25)
plt.show()

In [ ]:
print SSE_values

In [ ]:
U, s, V = np.linalg.svd(senti_matrix, full_matrices=False)

In [ ]:
print U.shape,  np.diag(s).shape, V.shape
print s

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=5)
low_dim_senti_matrix = pca.fit_transform(senti_matrix)
print low_dim_senti_matrix.shape

In [ ]:
variances = pca.explained_variance_ratio_
print variances[:5]

In [ ]:
principals=  2.5 + 5*pca.components_
#principals=  pca.components_
print principals[:5]

In [ ]:
#Trying to cluster in the SVD space 

low_dim_centroid_list = []
low_dim_id_list = []
for i in range(1 , 12):
    centroids,_ = kmeans(low_dim_senti_matrix,i)
    idx,_ = vq(low_dim_senti_matrix,centroids)
    low_dim_centroid_list.append(centroids)
    low_dim_id_list.append(idx)
    
low_dim_SSE_values = np.zeros(len(low_dim_id_list))
for i in range(len(low_dim_centroid_list)):
    for j in range(len(low_dim_centroid_list[i])):
        vecs = low_dim_senti_matrix[id_list[i]==j,:]
        #print vecs.shape
        cent = low_dim_centroid_list[i][j]
        SSE_1 = 0.0
        for vec in vecs:
            SSE_1 = SSE_1 + minkowski(vec,cent,2)
        low_dim_SSE_values[j] = low_dim_SSE_values[j] + SSE_1
for i in range(len(low_dim_SSE_values)):
    low_dim_SSE_values[i] = low_dim_SSE_values[i]/(i+1)

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(15, 15)
plt.rc('xtick', labelsize=20) 
plt.rc('ytick', labelsize=20)
x = np.arange(1,11)
plt.plot(x,low_dim_SSE_values[:10] ,linewidth = 3.0)
plt.xlabel("Number of Clusters", fontsize = 25)
plt.ylabel("Mean Minkowski distance from cluster centroids", fontsize = 25)
plt.title("Cluster groupings for Sentiment vectors for Videos", fontsize = 25)
plt.show()

In [ ]:
low_dim_centroid_final,_ = kmeans(low_dim_senti_matrix,3)
low_dim_idx_final,_ = vq(low_dim_senti_matrix,low_dim_centroid_final)

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(20, 15)
plt.rc('xtick', labelsize=20) 
plt.rc('ytick', labelsize=20)
plt.title("Sentiment transition graphs for all cluster centroids in Latent PC space", fontsize = 25)
plt.plot(low_dim_centroid_final[0], linewidth = 2.0)
plt.plot(low_dim_centroid_final[1], linewidth = 2.0)
plt.plot(low_dim_centroid_final[2], linewidth = 2.0)
plt.plot(low_dim_centroid_final[3], linewidth = 2.0)
#plt.plot(centroid_final[4], linewidth = 2.0 )
# plt.plot(centroid_final[5], linewidth = 2.0 )
# plt.plot(centroid_final[6], linewidth = 2.0 )

plt.xlabel("nth second into the video", fontsize = 25)
plt.ylabel("Sentiment", fontsize = 25)
plt.legend(['Cluster 1', 'Cluster 2' , 'Cluster 3' 
            ,'Cluster 4'
            ,'Cluster 5'
            #,'Cluster 6'
            #,'Cluster 7'
            
           ])
plt.show()

In [ ]:
# S = np.diag(s)
# Mhat = np.dot(U, np.dot(S, V.T))
# print Mhat.shape

In [ ]:
# print "Using all PCs, MSE = %.6G" %(np.mean((senti_matrix - Mhat)**2)/len(senti_matrix))

In [ ]:
# Mhat2 = np.dot(U[:, :1], np.dot(S[:1, :1], V[:,:1].T))
# print Mhat2.shape
# print "Using first 20 PCs, MSE = %.6G" %(np.mean((senti_matrix - Mhat2)**2)/len(senti_matrix))

In [ ]:
# norms = np.linalg.norm(Modes,axis=1)

# indexlist = np.argsort(np.linalg.norm(Modes,axis=1))
# sorted_modes = Modes[indexlist]
# norms = np.linalg.norm(sorted_modes,axis=1)
# plt.plot(s)
# plt.show()

In [ ]:
#print sorted_modes[:3]

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(20, 15)
plt.rc('xtick', labelsize=20) 
plt.rc('ytick', labelsize=20)
plt.title("Plot of top 5 left Singular Vectors", fontsize = 25)
plt.plot(principals[0], linewidth = 2.0)
plt.plot(principals[1], linewidth = 2.0)
plt.plot(principals[2], linewidth = 2.0)
plt.plot(principals[3], linewidth = 2.0)
plt.plot(principals[4], linewidth = 2.0 )
#plt.plot(principals[5], linewidth = 2.0 )
#plt.plot(principals[6], linewidth = 2.0 )
#plt.plot(sorted_modes[7,1:], linewidth = 2.0 )
#plt.plot(principals[8,1:], linewidth = 2.0 )

plt.xlabel("nth second into the video", fontsize = 25)
plt.ylabel("Sentiment", fontsize = 25)
plt.legend(['Cluster 1', 'Cluster 2' , 'Cluster 3' 
            ,'Cluster 4'
            ,'Cluster 5'
            #,'Cluster 6'
            #,'Cluster 7'
            
           ])
plt.show()

In [ ]:
index1 , topSentis1 = topNColliniear(principals[0], senti_matrix , 3) 
videosBasis1 , ANP1  = filterCluster(postIdFilteredList,filteredPosts,filteredDict,index1)

index2 , topSentis2 = topNColliniear(principals[1], senti_matrix , 3) 
videosBasis2 , ANP2  = filterCluster(postIdFilteredList,filteredPosts,filteredDict,index2)

index3 , topSentis3 = topNColliniear(principals[2], senti_matrix , 3) 
videosBasis3 , ANP3  = filterCluster(postIdFilteredList,filteredPosts,filteredDict,index3)

index4 , topSentis4 = topNColliniear(principals[3], senti_matrix , 3) 
videosBasis4 , ANP4  = filterCluster(postIdFilteredList,filteredPosts,filteredDict,index4)

index5 , topSentis5 = topNColliniear(principals[4], senti_matrix , 3) 
videosBasis5 , ANP5  = filterCluster(postIdFilteredList,filteredPosts,filteredDict,index5)

In [ ]:
print ANP2

In [ ]:
selectedid =  videosBasis5[2]['postId']
print megaDict[selectedid]
videoSentimentIndices = megaDict[selectedid]['indexList']
for i in range(len(videoSentimentIndices)):
    a = vineList[videoSentimentIndices[i]].split(',')
    print "Image %d :  %s" %(i , str(a))

In [ ]:
# for k in ANP1:
#     indexes = k['indexList']
#     for i in indexes:
#         print vineList[i]

In [ ]:
print (180/3.142)*0.034

In [ ]:
import io
import base64
from IPython.display import HTML

num = 2
url = videosBasis5[num]['videoUrl'].strip().split('/')[-1].split('?')[0]
print  videosBasis5[num]['videoUrl']
#print videosBasis1[num]
#print url
videoPath = "/datasets/sagarj/vineData/Dataset/Videos/" + url
#print videoPath


video = io.open(videoPath, 'r+b').read()
encoded = base64.b64encode(video)
HTML(data='''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii')))

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(20, 15)
plt.rc('xtick', labelsize=20) 
plt.rc('ytick', labelsize=20)
plt.title("Sentiment Plot of top collinear sentiment vectors for all basis", fontsize = 25)
plt.plot(topSentis1[0], linewidth = 2.0 , linestyle='dashed')
plt.plot(topSentis2[0], linewidth = 2.0 , linestyle='dashed')
plt.plot(topSentis3[0], linewidth = 2.0 , linestyle='dashed')
plt.plot(topSentis4[0], linewidth = 2.0 , linestyle='dashed')
plt.plot(topSentis5[0], linewidth = 2.0 , linestyle='dashed')

plt.xlabel("nth second into the video", fontsize = 25)
plt.ylabel("Sentiment", fontsize = 25)
plt.legend(['Basis 1', 'Basis 2' , 'Basis 3' 
            ,'Basis 4'
            ,'Basis 5'
            #,'Cluster 6'
            #,'Cluster 7'
            
           ])
plt.show()

In [ ]:
centroid_final,_ = kmeans(senti_matrix,4)
idx_final,_ = vq(senti_matrix,centroid_final)

In [ ]:
vecs1 = senti_matrix[idx_final==0,:]
vecs2 = senti_matrix[idx_final==1,:]
vecs3 = senti_matrix[idx_final==2,:]
vecs4 = senti_matrix[idx_final==3,:]
# vecs5 = senti_matrix[idx_final==4,:]
# vecs6 = senti_matrix[idx_final==5,:]
# vecs7 = senti_matrix[idx_final==6,:]
#senti_matrix[idx_final==0]

In [ ]:
print len(vecs1)
print len(vecs2)
print len(vecs3)
print len(vecs4)
# print len(vecs5)
# print len(vecs6)
# print len(vecs7)

In [ ]:
print centroid_final

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(20, 15)
plt.rc('xtick', labelsize=20) 
plt.rc('ytick', labelsize=20)
plt.title("Sentiment transition graphs for all cluster centroids", fontsize = 25)
plt.plot(centroid_final[0], linewidth = 2.0)
plt.plot(centroid_final[1], linewidth = 2.0)
plt.plot(centroid_final[2], linewidth = 2.0)
plt.plot(centroid_final[3], linewidth = 2.0)
#plt.plot(centroid_final[4], linewidth = 2.0 )
# plt.plot(centroid_final[5], linewidth = 2.0 )
# plt.plot(centroid_final[6], linewidth = 2.0 )

plt.xlabel("nth second into the video", fontsize = 25)
plt.ylabel("Sentiment", fontsize = 25)
plt.legend(['Cluster 1', 'Cluster 2' , 'Cluster 3' 
            ,'Cluster 4'
            ,'Cluster 5'
            #,'Cluster 6'
            #,'Cluster 7'
            
           ])
plt.show()

In [ ]:
index1_clusters , topSentis1_clusters = topNColliniear(centroid_final[0], senti_matrix[idx_final==0,:] , 100) 
videosBasis1_clusters , ANP1_clusters  = filterCluster(postIdFilteredList,filteredPosts,filteredDict,index1_clusters)

index2_clusters , topSentis2_clusters = topNColliniear(centroid_final[1], senti_matrix[idx_final==1,:] , 100) 
videosBasis2_clusters , ANP2_clusters  = filterCluster(postIdFilteredList,filteredPosts,filteredDict,index2_clusters)

index3_clusters , topSentis3_clusters = topNColliniear(centroid_final[2], senti_matrix[idx_final==2,:] , 100) 
videosBasis3_clusters , ANP3_clusters  = filterCluster(postIdFilteredList,filteredPosts,filteredDict,index3_clusters)

index4_clusters , topSentis4_clusters = topNColliniear(centroid_final[3], senti_matrix[idx_final==3,:] , 100) 
videosBasis4_clusters , ANP4_clusters  = filterCluster(postIdFilteredList,filteredPosts,filteredDict,index4_clusters)

# index5_clusters , topSentis5_clusters = topNColliniear(centroid_final[4], senti_matrix , 5) 
# videosBasis5_clusters , ANP5_clusters  = filterCluster(postIdFilteredList,filteredPosts,filteredDict,index5_clusters)

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(20, 15)
plt.rc('xtick', labelsize=20) 
plt.rc('ytick', labelsize=20)
plt.title("Sentiment Plot of top collinear sentiment vectors for all Cluster centroids", fontsize = 25)
plt.plot(topSentis1_clusters[50], linewidth = 2.0 , linestyle='dashed')
plt.plot(topSentis2_clusters[50], linewidth = 2.0 , linestyle='dashed')
plt.plot(topSentis3_clusters[50], linewidth = 2.0 , linestyle='dashed')
plt.plot(topSentis4_clusters[50], linewidth = 2.0 , linestyle='dashed')
#plt.plot(topSentis5_clusters[4], linewidth = 2.0 , linestyle='dashed')

plt.xlabel("nth second into the video", fontsize = 25)
plt.ylabel("Sentiment", fontsize = 25)
plt.legend(['Centroid for C1', 'Centroid for C2' , 'Centroid for C3' 
            ,'Centroid for C4'
            ,'Centroid for C5'
            #,'Cluster 6'
            #,'Cluster 7'
            
           ])
plt.show()

In [ ]:
selectedid =  videosBasis4_clusters[2]['postId']
print megaDict[selectedid]
videoSentimentIndices = megaDict[selectedid]['indexList']
for i in range(len(videoSentimentIndices)):
    a = vineList[videoSentimentIndices[i]].split(',')
    print "Image %d :  %s" %(i , str(a))

In [ ]:
import io
import base64
from IPython.display import HTML

num = 2
url = videosBasis4_clusters[num]['videoUrl'].strip().split('/')[-1].split('?')[0]
print  videosBasis4_clusters[num]['videoUrl']
print  videosBasis4_clusters[num]['postId']
#print videosBasis1[num]
#print url
videoPath = "/datasets/sagarj/vineData/Dataset/Videos/" + url
#print videoPath


video = io.open(videoPath, 'r+b').read()
encoded = base64.b64encode(video)
HTML(data='''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii')))

In [ ]:
index1_clusters , topSentis1_clusters = topNColliniear(centroid_final[0], senti_matrix[idx_final==0,:] , 100) 
videos1_clusters , ANP1_clusters  = filterCluster(postIdFilteredList,filteredPosts,filteredDict,index1_clusters)

index2_clusters , topSentis2_clusters = topNColliniear(centroid_final[1], senti_matrix[idx_final==1,:] , 100) 
videos2_clusters , ANP2_clusters  = filterCluster(postIdFilteredList,filteredPosts,filteredDict,index2_clusters)

index3_clusters , topSentis3_clusters = topNColliniear(centroid_final[2], senti_matrix[idx_final==2,:] , 100) 
videos3_clusters , ANP3_clusters  = filterCluster(postIdFilteredList,filteredPosts,filteredDict,index3_clusters)

index4_clusters , topSentis4_clusters = topNColliniear(centroid_final[3], senti_matrix[idx_final==3,:] , 100) 
videos4_clusters , ANP4_clusters  = filterCluster(postIdFilteredList,filteredPosts,filteredDict,index4_clusters)

In [ ]:
selectedid =  videos1_clusters[2]['postId']
print megaDict[selectedid]
videoSentimentIndices = megaDict[selectedid]['indexList']
for i in range(len(videoSentimentIndices)):
    a = vineList[videoSentimentIndices[i]].split(',')
    print "Image %d :  %s" %(i , str(a))

In [ ]:
def filterCluster(postids , posts , postDict, clusterMems , clusterId ):
    plist = []
    ANP_list = []
    selective = clusterMems == clusterId
    ids = [i for (i, v) in zip(postids, selective) if v]
    for postid in ids:
        plist.append(posts[postid])
        ANP_list.append(postDict[postid])
    return plist , ANP_list

def likesRepostsLoops(cluster):
    likes = np.zeros(len(cluster))
    reposts = np.zeros(len(cluster))
    loops = np.zeros(len(cluster))
    for i in range(len(cluster)):
        likes[i] = np.log(cluster[i]['likes']['count']+1)
        reposts[i] = np.log(cluster[i]['reposts']['count']+1)
        loops[i] = np.log(cluster[i]['loops']['count']+1)
    return likes, reposts, loops


In [ ]:
cluster1 , ANP_cluster1 = filterCluster(postIdFilteredList,filteredPosts,filteredDict,idx_final,0)
print len(cluster1)
cluster2 , ANP_cluster2 = filterCluster(postIdFilteredList,filteredPosts,filteredDict,idx_final,1)
print len(cluster2)
cluster3 , ANP_cluster3 = filterCluster(postIdFilteredList,filteredPosts,filteredDict,idx_final,2)
print len(cluster3)
cluster4 , ANP_cluster4 = filterCluster(postIdFilteredList,filteredPosts,filteredDict,idx_final,3)
print len(cluster4)
cluster5 , ANP_cluster5 = filterCluster(postIdFilteredList,filteredPosts,filteredDict,idx_final,4)
print len(cluster5)

In [ ]:
likesC1 , repostsC1 , loopsC1 = likesRepostsLoops(cluster1)
likesC2 , repostsC2 , loopsC2 = likesRepostsLoops(cluster2)
likesC3 , repostsC3 , loopsC3 = likesRepostsLoops(cluster3)
likesC4 , repostsC4 , loopsC4 = likesRepostsLoops(cluster4)
likesC5 , repostsC5 , loopsC5 = likesRepostsLoops(cluster5)

In [ ]:
c1 = np.random.choice(len(likesC1), 300)
sampledLikesC1 = likesC1[c1]

c2 = np.random.choice(len(likesC2), 300)
sampledLikesC2 = likesC2[c2]

c3 = np.random.choice(len(likesC3), 300)
sampledLikesC3 = likesC3[c3]

c4 = np.random.choice(len(likesC4), 300)
sampledLikesC4 = likesC4[c4]

# c5 = np.random.choice(len(likesC5), 300)
# sampledLikesC5 = likesC5[c5]


In [ ]:
print "Numbers for Cluster C1: Median Reposts: %f , Mean Reposts: %f , Median Likes: %f , \
Mean Likes: %f , Median Loops: %f , Mean Loops \
%f"%(np.median(repostsC1),np.mean(repostsC1),np.median(sampledLikesC1),np.mean(sampledLikesC1),np.median(loopsC1),np.mean(loopsC1))

print "Numbers for Cluster C2: Median Reposts: %f , Mean Reposts: %f , Median Likes: %f , \
Mean Likes: %f , Median Loops: %f , Mean Loops \
%f"%(np.median(repostsC2),np.mean(repostsC2),np.median(sampledLikesC2),np.mean(sampledLikesC2),np.median(loopsC2),np.mean(loopsC2))

print "Numbers for Cluster C3: Median Reposts: %f , Mean Reposts: %f , Median Likes: %f , \
Mean Likes: %f , Median Loops: %f , Mean Loops \
%f"%(np.median(repostsC3),np.mean(repostsC3),np.median(sampledLikesC3),np.mean(sampledLikesC3),np.median(loopsC3),np.mean(loopsC3))

print "Numbers for Cluster C4: Median Reposts: %f , Mean Reposts: %f , Median Likes: %f , \
Mean Likes: %f , Median Loops: %f , Mean Loops \
%f"%(np.median(repostsC4),np.mean(repostsC4),np.median(sampledLikesC4),np.mean(sampledLikesC4),np.median(loopsC4),np.mean(loopsC4))

# print "Numbers for Cluster C5: Median Reposts: %f , Mean Reposts: %f , Median Likes: %f , \
# Mean Likes: %f , Median Loops: %f , Mean Loops \
# %f"%(np.median(repostsC5),np.mean(repostsC5),np.median(sampledLikesC5),np.mean(sampledLikesC5),np.median(loopsC5),np.mean(loopsC5))

In [ ]:
clf = LinearSVR()

clf.fit(vecs3, repostsC3) 
print clf.coef_

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(15, 10)
plt.hist(sampledLikesC1, 100, normed=1,histtype='step', cumulative=True,linewidth = 1.0)
plt.hist(sampledLikesC2, 100, normed=1,histtype='step', cumulative=True,linewidth = 1.0)
plt.hist(sampledLikesC3, 100, normed=1,histtype='step', cumulative=True, linewidth = 1.0)
plt.hist(sampledLikesC4, 100, normed=1,histtype='step', cumulative=True, linewidth = 1.0)
# plt.hist(sampledLikesC5, 100, normed=1,histtype='step', cumulative=True, linewidth = 1.0)
plt.title("CDF for Likes for the 4 sentiment clusters ", fontsize = 25)
plt.xlabel("Likes/repost count",fontsize = 25)
plt.ylabel("CDF",fontsize = 25)
plt.legend(['Likes C1', 'Likes C2' , 'Likes C3' , 'Likes C4' , 'Likes C5'])
plt.show()

In [ ]:

def plotDist(RV, samples, b):
    N = samples 
    n = b
    s = np.random.normal(size=N)   # generate your data sample with N elements
    p1, x1 = np.histogram(RV, bins=n) # bin it into n = N/10 bins
    xs1 = x1[:-1] + (x1[1] - x1[0])/2
    f1 = UnivariateSpline(xs1, p1)
    plt.plot(xs1 , f1(xs1) ,'-')

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(15, 10)
plotDist(sampledLikesC1 , 200 , 50)
plotDist(sampledLikesC2 , 200 , 50)
plotDist(sampledLikesC3 , 200 , 50)
plotDist(sampledLikesC4 , 200 , 50)
# plotDist(sampledLikesC5 , 200 , 50)
plt.legend(['Reposts C1', 'Reposts C2' , 'Reposts C3' , 'Reposts C4', 'Reposts C4'])
plt.xlabel("Like count per video per cluster in Log scale",fontsize = 25)
plt.ylabel("Frequency ",fontsize = 25)
plt.show()

In [ ]:
import io
import base64
from IPython.display import HTML

num = 155
url = cluster4[num]['videoDashUrl'].strip().split('/')[-1].split('?')[0]
print  cluster4[num]['videoDashUrl']
print ANP_cluster4[num]

#print url
videoPath = "/datasets/sagarj/vineData/Dataset/Videos/" + url
print videoPath


video = io.open(videoPath, 'r+b').read()
encoded = base64.b64encode(video)
HTML(data='''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii')))